In [11]:
#import requiered libraries
import os
import warnings
import sys

import dvc.api
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet, LogisticRegression
from sklearn.preprocessing import LabelEncoder
from fast_ml.model_development import train_valid_test_split
from sklearn.tree import DecisionTreeClassifier
from urllib.parse import urlparse


In [12]:
import warnings
warnings.filterwarnings('ignore')

In [13]:
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from mllearn import Ml
from preprocess import Preprocess

ModuleNotFoundError: No module named 'ml'